## ChromaDB connection and load data 

load data

In [7]:
cl = cs.get_specifics("PPS")
print(cl)

{'ids': ['id0', 'id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7'], 'embeddings': array([[-0.05616511, -0.01541862, -0.02970634, ...,  0.016752  ,
         0.00894882, -0.00475067],
       [-0.02731149,  0.00175687, -0.03545567, ...,  0.01454608,
        -0.05446029, -0.02225705],
       [-0.0324773 ,  0.01229401, -0.01113146, ..., -0.01004309,
        -0.05448178, -0.01861655],
       ...,
       [-0.03238031,  0.01179582, -0.0283231 , ..., -0.01092045,
        -0.01374846,  0.01287064],
       [-0.05800085,  0.01502216, -0.04128072, ..., -0.00755922,
        -0.03570972, -0.00576454],
       [-0.01509039,  0.00757372,  0.01312466, ..., -0.02653653,
        -0.02884256, -0.03378146]]), 'metadatas': [{'column_title': 'О конференции'}, {'column_title': 'Программа конференции'}, {'column_title': 'Секции__Секция 1'}, {'column_title': 'Секции__Секция 2'}, {'column_title': 'Секции__Секция 3'}, {'column_title': 'Секции__Секция 4'}, {'column_title': 'Требования__Требования к статье'}, {'column_

In [8]:
cqq = cs.find_similar_records("PPS", ["Статья должна быть оформлена"])

### Refactoring

In [5]:
# Создаем экземпляр ChromaDBManager
chroma_manager = ChromaDBManager()

# Получаем список всех коллекций
collections = chroma_manager.list_collections()

# Выводим только имена коллекций
collection_names = [collection.name for collection in collections]
print(f"Найдено {len(collection_names)} коллекций:")
print(", ".join(collection_names))

2025-04-21 18:36:28.689 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-21 18:36:28.690 | INFO     | __main__:__init__:18 - Initializing ChromaDBManager with host: localhost, port: 8080
2025-04-21 18:36:29.162 | INFO     | __main__:__init__:23 - Successfully connected to ChromaDB
2025-04-21 18:36:48.427 | INFO     | __main__:__init__:37 - Created embedding function with model: BAAI/bge-m3
2025-04-21 18:36:48.433 | INFO     | __main__:list_collections:51 - Listed 5 collections in ChromaDB


Найдено 5 коллекций:
KMU, FRUCT, EPI, YSC, PPS


In [10]:
# Удаляем все коллекции
#result = chroma_manager.delete_all_collections()

2025-04-18 22:19:53.492 | INFO     | __main__:list_collections:51 - Listed 1 collections in ChromaDB
2025-04-18 22:19:53.507 | INFO     | __main__:delete_all_collections:268 - Deleted collection 'PPS'
2025-04-18 22:19:53.508 | INFO     | __main__:delete_all_collections:286 - Delete all collections operation completed: 1 deleted, 0 failed


In [11]:
# Добавляем данные из Excel в коллекцию
# specifics_path = "D:/GitHub/Edulytica/src/rag/data/Specification.xlsx"
# chroma_manager.add_from_excel(specifics_path, "ППС", "PPS")
# chroma_manager.add_from_excel(specifics_path, "КМУ", "KMU")
# chroma_manager.add_from_excel(specifics_path, "ЭПИ", "EPI")
# chroma_manager.add_from_excel(specifics_path, "YSC", "YSC")
# chroma_manager.add_from_excel(specifics_path, "FRUCT", "FRUCT")









2025-04-18 22:22:17.948 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-18 22:22:17.948 | INFO     | __main__:__init__:15 - Initializing EmbeddingProcessor with model: BAAI/bge-m3
2025-04-18 22:22:17.948 | INFO     | __main__:__init__:20 - Successfully initialized embedding function with model BAAI/bge-m3
2025-04-18 22:22:17.949 | INFO     | __main__:process_excel_data:45 - Processing Excel data from file: D:/GitHub/Edulytica/src/rag/data/Specification.xlsx, sheet: ППС
2025-04-18 22:22:17.963 | INFO     | __main__:process_excel_data:51 - Successfully read Excel data with shape: (1, 8)
2025-04-18 22:22:17.963 | INFO     | __main__:process_excel_data:92 - Processed 8 documents from Excel data
2025-04-18 22:22:18.019 | INFO     | __main__:create_collection:82 - Created new collection 'PPS' with metadat

True

### Get data

In [15]:
import numpy as np

# №1 Chunks

1. Clear text
2. Simple chanks

In [16]:
# 1.2. Функция для разбиения текста на чанки
def split_text_to_chunks(text: str,
                         max_chunk_size: int = 500,
                         overlap: int = 50) -> List[str]:
    words = text.split()
    chunks = []
    step = max_chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = " ".join(words[i: i + max_chunk_size])
        chunks.append(chunk)
    return chunks

# Получаем список чанков
chunks = split_text_to_chunks(article_text)

In [17]:
# 2.1. Вычисляем эмбеддинги для каждого чанка
processor = EmbeddingProcessor()
embed_fn = processor.get_embedding_function()  # SentenceTransformerEmbeddingFunction
chunk_embeddings = embed_fn(chunks)

2025-04-21 19:17:41.098 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-21 19:17:41.099 | INFO     | __main__:__init__:15 - Initializing EmbeddingProcessor with model: BAAI/bge-m3
2025-04-21 19:17:41.100 | INFO     | __main__:__init__:20 - Successfully initialized embedding function with model BAAI/bge-m3


In [18]:
# 3.1. Получаем нужную коллекцию из ChromaDB
chroma_manager = ChromaDBManager()
collection = chroma_manager.get_collection("KMU")
# Загружаем из коллекции всё, что нужно:
contents = collection.get(include=['embeddings', 'documents', 'metadatas'])
col_embs = np.array(contents['embeddings'])        # (N_docs, dim)
docs     = contents['documents']                   # List[str]
mds      = contents['metadatas']  

2025-04-21 19:17:44.256 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-21 19:17:44.257 | INFO     | __main__:__init__:18 - Initializing ChromaDBManager with host: localhost, port: 8080
2025-04-21 19:17:44.494 | INFO     | __main__:__init__:23 - Successfully connected to ChromaDB
2025-04-21 19:17:44.496 | INFO     | __main__:__init__:37 - Created embedding function with model: BAAI/bge-m3
2025-04-21 19:17:44.498 | INFO     | __main__:get_collection:101 - Retrieved collection 'KMU'


In [19]:
# Нормализуем эмбеддинги коллекции:
col_embs_norm   = col_embs / np.linalg.norm(col_embs, axis=1, keepdims=True)


In [20]:
chunk_embs    = np.array(chunk_embeddings)         # (N_chunks, dim)
chunk_embs_norm = chunk_embs / np.linalg.norm(chunk_embs, axis=1, keepdims=True)

In [21]:
# косинусная похожесть: матричное умножение
sims_matrix = chunk_embs_norm.dot(col_embs_norm.T)  # (N_chunks, N_docs)
top_k = 5

In [22]:
results = []
for chunk_idx, sims in enumerate(sims_matrix):
    # индексы top_k документов для данного чанка
    best_idx = np.argsort(-sims)[:top_k]
    for doc_idx in best_idx:
        results.append({
            'chunk': chunk_idx,
            'similarity': float(sims[doc_idx]),
            'text': docs[doc_idx],
            'metadata': mds[doc_idx]
        })

In [23]:
# отсортируем все найденные пары (чанк,док) по убыванию схожести
results = sorted(results, key=lambda x: -x['similarity'])

# Выведем топ‑10
for r in results[:10]:
    print(f"[chunk {r['chunk']}] sim={r['similarity']:.4f} meta={r['metadata']}")
    print(r['text'])
    print('-'*60)

[chunk 0] sim=0.5526 meta={'column_title': 'Секции__Будущее экономики в эпоху технологических изменений: роль искусственного интеллекта и инноваций__ФИРМЫ И РЫНКИ: IT-решения для бизнеса / FIRM AND MARKETS: IT-driven solutions'}
Секция посвящена исследованиям в области инновационных IT-решений, цифровой трансформации и устойчивого развития бизнеса.
В центре внимания — технологии больших данных, искусственного интеллекта, организационных инноваций, автоматизации и блокчейна, а также их роль в повышении эффективности управления компаниями. Участники обсудят примеры внедрения цифровых решений в различных секторах экономики, включая ритейл, финансы и производство.
Исследования затрагивают аспекты взаимодействия IT-инноваций с организационной культурой и стратегиями компаний. Особое внимание будет уделено цифровым экосистемам, платформенным бизнес-моделям и новым инструментам анализа поведения потребителей.
------------------------------------------------------------
[chunk 0] sim=0.5443 me

# № 2  All text


In [24]:
article_emb = embed_fn([article_text])[0]          # (dim,)
article_emb_norm = article_emb / np.linalg.norm(article_emb)

# вычисляем sims к каждому документу:
sims = col_embs_norm.dot(article_emb_norm)         # (N_docs,)
best_idx = np.argsort(-sims)[:top_k]

print("\n=== Результаты поиска по всему тексту статьи ===")
for idx in best_idx:
    print(f"sim={sims[idx]:.4f} meta={mds[idx]}")
    print(docs[idx])
    print('-'*60)


=== Результаты поиска по всему тексту статьи ===
sim=0.5159 meta={'column_title': 'Секции__Искусственный интеллект__Системы искусственного интеллекта и больших данных'}
Секция соответствует магистерским программам "Проектирование и разработка систем искусственного интеллекта" и "Проектирование и разработка систем больших данных" по направлению подготовки "09.04.04 Программная инженерия", реализуемым факультетом ПИиКТ.

Направления работы секции:
Архитектура систем обработки больших данных
Хранение и алгоритмы сжатия данных
Структуры и алгоритмы индексации данных
Надежность систем хранения данных
Параллельная и распределенная обработка больших данных NoSQL хранилища данных
Архитектура систем искусственного интеллекта
Продвинутое машинное обучение и глубокое обучение
Обработка изображений и компьютерное зрение
Обработка звука и речи
Анализ текстов
Анализ потоковых данных
Распределенное машинное обучение MLOps
Валидация и тестирование систем искусственного интеллекта.
Языки секции: русск

# №3 Chunks + Keywords 

In [25]:
from typing import List
import re
from collections import Counter

import nltk
nltk.download('stopwords')  # если ещё не скачали стоп‑слова
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fortn/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [26]:
# 1) Функция для извлечения ключевых слов из произвольного текста
def extract_keywords(text: str, top_k: int = 10) -> List[str]:
    russian_stopwords = set(stopwords.words('russian'))
    # собираем все «слова» длиной >3, убираем стоп‑слова
    words = re.findall(r'\w+', text.lower())
    filtered = [w for w in words if w not in russian_stopwords and len(w) > 3]
    # считаем частоты и берём top_k
    freq = Counter(filtered)
    return [w for w, _ in freq.most_common(top_k)]

In [27]:
# 2) Извлекаем ключевые слова для каждого чанка
chunk_keywords = [extract_keywords(chunk, top_k=8) for chunk in chunks]

# 3) Подгружаем из коллекции все документы + эмбеддинги + метаданные
contents = collection.get(include=['embeddings', 'documents', 'metadatas'])
col_embs = np.array(contents['embeddings'])    # shape (N_docs, dim)
docs     = contents['documents']               # List[str]
mds      = contents['metadatas']       

In [29]:
# Нормализуем эмбеддинги коллекции
col_embs_norm = col_embs / np.linalg.norm(col_embs, axis=1, keepdims=True)

results = []


In [30]:
# 4) Для каждого чанка:
for idx_chunk, (chunk_emb, kws) in enumerate(zip(chunk_embeddings, chunk_keywords)):
    # 4.1) находим из коллекции документы, где есть хотя бы одно ключевое слово
    matching_idx = [
        i for i, doc in enumerate(docs)
        if any(kw in doc.lower() for kw in kws)
    ]
    if not matching_idx:
        # если ничего не нашли по ключевым словам, используем всю коллекцию
        matching_idx = list(range(len(docs)))

    # готовим подмножество эмбеддингов и текстов
    subset_embs_norm = col_embs_norm[matching_idx]
    subset_docs      = [docs[i] for i in matching_idx]
    subset_mds       = [mds[i] for i in matching_idx]

    # нормализуем текущий чанк
    emb_norm = chunk_emb / np.linalg.norm(chunk_emb)

    # 4.2) считаем косинусную похожесть
    sims = subset_embs_norm.dot(emb_norm)  # (len(matching_idx),)

    # 4.3) берём top_k по сходству
    top_k = 5
    best_pos = np.argsort(-sims)[:top_k]
    for pos in best_pos:
        orig_i = matching_idx[pos]
        results.append({
            'chunk_index': idx_chunk,
            'keywords':       kws,
            'similarity':     float(sims[pos]),
            'text':           subset_docs[pos],
            'metadata':       subset_mds[pos]
        })

# 5) Сортируем все результаты сразу
results = sorted(results, key=lambda x: -x['similarity'])

# 6) Выводим топ‑10
for r in results[:10]:
    print(f"[chunk {r['chunk_index']}] sim={r['similarity']:.4f} keys={r['keywords']}")
    print(r['metadata'])
    print(r['text'])
    print('-'*60)

[chunk 0] sim=0.5526 keys=['данных', 'блоков', 'хранения', 'системы', 'систем', 'распределенных', 'реестров', 'блока']
{'column_title': 'Секции__Будущее экономики в эпоху технологических изменений: роль искусственного интеллекта и инноваций__ФИРМЫ И РЫНКИ: IT-решения для бизнеса / FIRM AND MARKETS: IT-driven solutions'}
Секция посвящена исследованиям в области инновационных IT-решений, цифровой трансформации и устойчивого развития бизнеса.
В центре внимания — технологии больших данных, искусственного интеллекта, организационных инноваций, автоматизации и блокчейна, а также их роль в повышении эффективности управления компаниями. Участники обсудят примеры внедрения цифровых решений в различных секторах экономики, включая ритейл, финансы и производство.
Исследования затрагивают аспекты взаимодействия IT-инноваций с организационной культурой и стратегиями компаний. Особое внимание будет уделено цифровым экосистемам, платформенным бизнес-моделям и новым инструментам анализа поведения потре

In [ ]:
# Добавляем данные из Excel в коллекцию
# specifics_path = "D:/GitHub/Edulytica/src/rag/data/Specification.xlsx"
# chroma_manager.add_from_excel(specifics_path, "ППС", "PPS")
# chroma_manager.add_from_excel(specifics_path, "КМУ", "KMU")
# chroma_manager.add_from_excel(specifics_path, "ЭПИ", "EPI")
# chroma_manager.add_from_excel(specifics_path, "YSC", "YSC")
# chroma_manager.add_from_excel(specifics_path, "FRUCT", "FRUCT")

# Общий код 

In [1]:
import sys
import os
import numpy as np
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
import chromadb
from copy import deepcopy
import os
from typing import List, Dict, Any, Optional, Union, Tuple
from loguru import logger
from rag.core.utils.config_loader import ConfigLoader

In [2]:
class EmbeddingProcessor:
    """
    Класс для создания и обработки эмбеддингов из текстовых данных
    """
    def __init__(self, embedding_model: Optional[str] = None):
        """
        Инициализация с выбором модели эмбеддингов
        
        Args:
            embedding_model: Название модели для создания эмбеддингов (опционально)
                            Если не указана, будет использована модель из конфигурации
        """
        config_loader = ConfigLoader()
        self.embedding_model = embedding_model or config_loader.get_embedding_model()
        logger.info(f"Initializing EmbeddingProcessor with model: {self.embedding_model}")
        
        try:
            self.embedding_function = chromadb.utils.embedding_functions.SentenceTransformerEmbeddingFunction(
                model_name=self.embedding_model)
            logger.info(f"Successfully initialized embedding function with model {self.embedding_model}")
        except Exception as e:
            logger.error(f"Failed to initialize embedding function: {e}")
            raise
    
    def get_embedding_function(self):
        """
        Получить функцию для создания эмбеддингов
        
        Returns:
            Функция для создания эмбеддингов
        """
        return self.embedding_function
    
    def process_excel_data(self, file_name: str, sheet_name: str) -> Dict[str, Any]:
        """
        Обработка Excel-файла и подготовка данных для эмбеддингов
        
        Args:
            file_name: Путь к Excel-файлу
            sheet_name: Имя листа в Excel-файле
            
        Returns:
            Словарь со структурированными данными для сохранения в ChromaDB
        """
        logger.info(f"Processing Excel data from file: {file_name}, sheet: {sheet_name}")
        
        try:
            # Чтение данных из Excel
            spec_data = pd.read_excel(file_name, sheet_name=sheet_name, header=[0,1,2,3])
            spec_data = spec_data.fillna("")
            logger.info(f"Successfully read Excel data with shape: {spec_data.shape}")
            
            # Обработка данных
            res = {}
            res['description_full'] = []
            
            # Обрабатываем заголовки столбцов
            for i in spec_data.columns:
                ii = list(i)
                res['description_full'].append({})
                res['description_full'][-1]['column_title'] = ''
                
                for j in range(1, len(ii)):
                    if not (ii[j].startswith("Unnamed")):
                        res['description_full'][-1]['column_title'] += ii[j]
                        res['description_full'][-1]['column_title'] += '__'
                
                res['description_full'][-1]['column_title'] = res['description_full'][-1]['column_title'][:-2]
                res['description_full'][-1]['column_text'] = " ".join(spec_data[i].tolist()).strip()
            
            # Готовим данные для ChromaDB
            ids = []
            documents = []
            metadatas = []
            
            for j in range(len(res['description_full'])):
                ids.append(f"id{j}")
                documents.append(res['description_full'][j]['column_text'])
                md = {
                    'column_title': res['description_full'][j]['column_title']
                }
                metadatas.append(deepcopy(md))
            
            # Создаем метаданные коллекции
            collection_metadata = {
                "source_file": file_name,
                "sheet_name": sheet_name,
                "conf_title": spec_data.columns[0][0],
                "conf_title_short": sheet_name
            }
            
            logger.info(f"Processed {len(documents)} documents from Excel data")
            
            return {
                "ids": ids,
                "documents": documents,
                "metadatas": metadatas,
                "collection_metadata": collection_metadata
            }
            
        except Exception as e:
            logger.error(f"Error processing Excel data: {e}")
            raise


In [3]:
class ChromaDBManager:
    """
    Класс для управления коллекциями в ChromaDB
    """
    def __init__(self, embedding_function=None):
        """
        Инициализация менеджера ChromaDB и установление соединения
        
        Args:
            embedding_function: функция для создания эмбеддингов (опционально)
                              Если не указана, будет использована модель из конфигурации
        """
        config_loader = ConfigLoader()
        config = config_loader.load_config()
        self.host = config_loader.get_host()
        self.port = config_loader.get_port()
        
        logger.info(f"Initializing ChromaDBManager with host: {self.host}, port: {self.port}")
        
        # Устанавливаем соединение с ChromaDB
        try:
            self.chroma_client = chromadb.HttpClient(host=self.host, port=self.port)
            logger.info("Successfully connected to ChromaDB")
        except Exception as e:
            logger.error(f"Failed to connect to ChromaDB: {e}")
            raise
        
        # Используем переданную функцию эмбеддингов или создаем новую из конфигурации
        if embedding_function:
            self.embedding_function = embedding_function
            logger.info("Using provided embedding function")
        else:
            try:
                embedding_model = config_loader.get_embedding_model()
                self.embedding_function = chromadb.utils.embedding_functions.SentenceTransformerEmbeddingFunction(
                    model_name=embedding_model)
                logger.info(f"Created embedding function with model: {embedding_model}")
            except Exception as e:
                logger.error(f"Failed to create embedding function: {e}")
                raise
    
    def list_collections(self) -> List[Dict[str, Any]]:
        """
        Получить список всех коллекций в ChromaDB
        
        Returns:
            Список коллекций с их метаданными
        """
        try:
            collections = self.chroma_client.list_collections()
            logger.info(f"Listed {len(collections)} collections in ChromaDB")
            return collections
        except Exception as e:
            logger.error(f"Error listing collections: {e}")
            return []
    
    def create_collection(self, name: str, metadata: Optional[Dict[str, Any]] = None) -> Any:
        """
        Создать новую коллекцию
        
        Args:
            name: Имя коллекции
            metadata: Метаданные коллекции (опционально)
            
        Returns:
            Объект коллекции
        """
        try:
            # Пробуем получить существующую коллекцию
            collection = self.chroma_client.get_collection(
                name=name, 
                embedding_function=self.embedding_function
            )
            logger.info(f"Collection '{name}' already exists.")
        except Exception:
            # Если коллекция не существует, создаем новую
            collection = self.chroma_client.create_collection(
                name=name,
                metadata=metadata,
                embedding_function=self.embedding_function
            )
            logger.info(f"Created new collection '{name}' with metadata: {metadata}")
        
        return collection
    
    def get_collection(self, name: str) -> Any:
        """
        Получить коллекцию по имени
        
        Args:
            name: Имя коллекции
            
        Returns:
            Объект коллекции или None, если коллекция не найдена
        """
        try:
            collection = self.chroma_client.get_collection(
                name=name, 
                embedding_function=self.embedding_function
            )
            logger.info(f"Retrieved collection '{name}'")
            return collection
        except Exception as e:
            logger.error(f"Error getting collection '{name}': {e}")
            return None
    
   
    
    def add_documents(self, 
                     collection_name: str, 
                     documents: List[str], 
                     metadatas: Optional[List[Dict[str, Any]]] = None, 
                     ids: Optional[List[str]] = None) -> bool:
        """
        Добавить документы в коллекцию
        
        Args:
            collection_name: Имя коллекции
            documents: Список текстов документов
            metadatas: Список метаданных для каждого документа (опционально)
            ids: Список идентификаторов для каждого документа (опционально)
            
        Returns:
            True, если документы успешно добавлены, иначе False
        """
        try:
            collection = self.get_collection(collection_name)
            if not collection:
                logger.error(f"Collection '{collection_name}' not found")
                return False
            
            # Генерируем идентификаторы, если они не предоставлены
            if not ids:
                ids = [f"id{i}" for i in range(len(documents))]
            
            # Добавляем документы в коллекцию
            collection.add(
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )
            
            logger.info(f"Added {len(documents)} documents to collection '{collection_name}'")
            return True
        except Exception as e:
            logger.error(f"Error adding documents to collection '{collection_name}': {e}")
            return False
    

    
    def get_collection_contents(self, 
                              collection_name: str, 
                              include: Optional[List[str]] = None) -> Dict[str, Any]:
        """
        Получить все содержимое коллекции
        
        Args:
            collection_name: Имя коллекции
            include: Что включать в результаты ("documents", "metadatas", "embeddings")
            
        Returns:
            Содержимое коллекции или пустой словарь в случае ошибки
        """
        try:
            collection = self.get_collection(collection_name)
            if not collection:
                logger.error(f"Collection '{collection_name}' not found")
                return {}
            
            # Устанавливаем, что включать в результаты по умолчанию
            if include is None:
                include = ["documents", "metadatas"]
            
            # Получаем все содержимое коллекции
            contents = collection.get(include=include)
            
            logger.info(f"Got contents for collection '{collection_name}': {len(contents.get('ids', []))} documents")
            return contents
        except Exception as e:
            logger.error(f"Error getting collection contents for '{collection_name}': {e}")
            return {}
    

    
    def add_from_excel(self, file_name: str, sheet_name: str, collection_name: str) -> bool:
        """
        Добавить данные из Excel-файла в коллекцию ChromaDB
        
        Args:
            file_name: Путь к Excel-файлу
            sheet_name: Имя листа в Excel-файле
            collection_name: Имя коллекции для создания/обновления
            
        Returns:
            True, если данные успешно добавлены, иначе False
        """
        try:
            # Создаем процессор эмбеддингов для обработки данных
            processor = EmbeddingProcessor()
            
            # Обрабатываем данные из Excel
            data = processor.process_excel_data(file_name, sheet_name)
            
            # Создаем или получаем коллекцию
            self.create_collection(
                name=collection_name, 
                metadata=data["collection_metadata"]
            )
            
            # Добавляем документы в коллекцию
            success = self.add_documents(
                collection_name=collection_name,
                documents=data["documents"],
                metadatas=data["metadatas"],
                ids=data["ids"]
            )
            
            if success:
                logger.info(f"Successfully added data from {file_name} ({sheet_name}) to collection '{collection_name}'")
            else:
                logger.error(f"Failed to add data from {file_name} to collection '{collection_name}'")
            
            return success
        except Exception as e:
            logger.error(f"Error adding data from Excel to collection: {e}")
            return False
        
        
    def delete_all_collections(self) -> Dict[str, Any]:
        """
        Удалить все коллекции в ChromaDB
        
        Returns:
            Словарь с информацией о результатах удаления:
            {
                "success": bool - общий успех операции,
                "total": int - общее количество коллекций,
                "deleted": int - количество успешно удаленных коллекций,
                "failed": int - количество коллекций, которые не удалось удалить,
                "failed_collections": List[str] - имена коллекций, которые не удалось удалить
            }
        """
        try:
            # Получаем список всех коллекций
            collections = self.list_collections()
            
            if not collections:
                logger.info("No collections found to delete")
                return {
                    "success": True,
                    "total": 0,
                    "deleted": 0,
                    "failed": 0,
                    "failed_collections": []
                }
            
            # Счетчики для отслеживания прогресса
            deleted_count = 0
            failed_count = 0
            failed_collections = []
            
            # Удаляем каждую коллекцию
            for collection in collections:
                collection_name = collection.name
                try:
                    self.chroma_client.delete_collection(name=collection_name)
                    deleted_count += 1
                    logger.info(f"Deleted collection '{collection_name}'")
                except Exception as e:
                    failed_count += 1
                    failed_collections.append(collection_name)
                    logger.error(f"Failed to delete collection '{collection_name}': {e}")
            
            # Определяем общий успех операции
            success = failed_count == 0
            
            # Результат операции
            result = {
                "success": success,
                "total": len(collections),
                "deleted": deleted_count,
                "failed": failed_count,
                "failed_collections": failed_collections
            }
            
            logger.info(f"Delete all collections operation completed: {deleted_count} deleted, {failed_count} failed")
            return result
            
        except Exception as e:
            logger.error(f"Error in delete_all_collections: {e}")
            return {
                "success": False,
                "total": 0,
                "deleted": 0,
                "failed": 0,
                "failed_collections": [],
                "error": str(e)
            }

In [4]:
chroma_manager = ChromaDBManager()

2025-04-22 19:14:09.418 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-22 19:14:09.419 | INFO     | __main__:__init__:18 - Initializing ChromaDBManager with host: localhost, port: 8080
2025-04-22 19:14:09.923 | INFO     | __main__:__init__:23 - Successfully connected to ChromaDB
2025-04-22 19:14:19.509 | INFO     | __main__:__init__:37 - Created embedding function with model: BAAI/bge-m3


In [5]:

# Получаем все содержимое коллекции "PPS"
kmu_data = chroma_manager.get_collection_contents(
    collection_name="KMU",
    include=["documents", "embeddings", "metadatas"]
)
kmu_data

2025-04-22 19:14:19.534 | INFO     | __main__:get_collection:101 - Retrieved collection 'KMU'
2025-04-22 19:14:19.635 | INFO     | __main__:get_collection_contents:177 - Got contents for collection 'KMU': 192 documents


{'ids': ['id0',
  'id1',
  'id2',
  'id3',
  'id4',
  'id5',
  'id6',
  'id7',
  'id8',
  'id9',
  'id10',
  'id11',
  'id12',
  'id13',
  'id14',
  'id15',
  'id16',
  'id17',
  'id18',
  'id19',
  'id20',
  'id21',
  'id22',
  'id23',
  'id24',
  'id25',
  'id26',
  'id27',
  'id28',
  'id29',
  'id30',
  'id31',
  'id32',
  'id33',
  'id34',
  'id35',
  'id36',
  'id37',
  'id38',
  'id39',
  'id40',
  'id41',
  'id42',
  'id43',
  'id44',
  'id45',
  'id46',
  'id47',
  'id48',
  'id49',
  'id50',
  'id51',
  'id52',
  'id53',
  'id54',
  'id55',
  'id56',
  'id57',
  'id58',
  'id59',
  'id60',
  'id61',
  'id62',
  'id63',
  'id64',
  'id65',
  'id66',
  'id67',
  'id68',
  'id69',
  'id70',
  'id71',
  'id72',
  'id73',
  'id74',
  'id75',
  'id76',
  'id77',
  'id78',
  'id79',
  'id80',
  'id81',
  'id82',
  'id83',
  'id84',
  'id85',
  'id86',
  'id87',
  'id88',
  'id89',
  'id90',
  'id91',
  'id92',
  'id93',
  'id94',
  'id95',
  'id96',
  'id97',
  'id98',
  'id99',
  '

In [7]:
# Получаем текст статьи
# Путь к файлу статьи
article_path = "D:/GitHub/Edulytica/src/rag/data/article.txt"

# Загружаем содержимое файла
with open(article_path, 'r', encoding='utf-8') as file:
    article_text = file.read()

article_text


"ДЛЯ РАСПРЕДЕЛЕННОГО ХРАНЕНИЯ ДАННЫХ \nКича И.В.1 (студент), Тимкин А.К.1 (студент) \nНаучный руководитель – к.т.н., доцент ФБИТ Таранов С.В. \n1 – Университет ИТМО \ne-mail: kicha@niuitmo.ru \nВ ходе данной работы были рассмотрены различные методы организации систем \nраспределенных реестров. Были рассмотрены примитивные структуры, древовидные и их \nмодификации. Была проанализирована производительность системы хранения данных при \nиспользовании разных подходов. Были предложены эффективные алгоритмы подтверждения \nналичия блока в цепи. \nКлючевые слова \nРаспределенные реестры, хранение данных, древовидные структуры данных, алгоритмы \nупрощенной проверки оплаты, блокчейн. \nПо данным различных исследования последнее десятилетие возрастает количество \nисследований в области систем распределенных реестров. Таким образом, число \nразработок увеличивается и их область применения расширяется. Одним из таких \nнаправлений является возможность хранения данных. Системы, основанные на \nте

# План:
1. Работа с получением общей информации о конференции, т.е её специфики 
2. Получение данных на основе текста статьи.
3. Получение данных на основе запроса.
   3.1 Полный промт
   3.2 Основаная часть промта с шагами
   3.3 Отдельно каждый шаг

## 1. Работа с получением общей инфомрации о конференции, т.е её специфики и т.д


### 1.1 Формирование запросов к коллекции

Мы формулируем список запросов (queries) по ключевым аспектам:

* «общая информация о конференции»
* «правила оформления статей»
* «требования к структуре и форматированию»

Процесс работы:
1. Вызываем у коллекции метод `query(query_texts=..., n_results=..., include=...)`
2. Для каждого запроса получаем `documents`, `metadatas` и `distances`
3. Собираем в словарь, где:
   - **Ключ**: сам запрос
   - **Значение**: список топ-K фрагментов с дистанцией

In [8]:
def load_collection_contents(
    manager: ChromaDBManager,
    collection_name: str
) -> Dict[str, Any]:
    """
    Загружает из коллекции эмбеддинги, документы и метаданные.
    """
    collection = manager.get_collection(collection_name)
    if collection is None:
        raise RuntimeError(f"Коллекция '{collection_name}' не найдена")
    
    contents = collection.get(include=['embeddings', 'documents', 'metadatas'])
    return {
        'embeddings': np.array(contents['embeddings']),  # shape (N_docs, dim)
        'documents':  contents['documents'],             # List[str]
        'metadatas':  contents['metadatas']              # List[dict]
    }

In [9]:
def normalize_embeddings(embs: np.ndarray) -> np.ndarray:
    """
    L2‑нормализует эмбеддинги по каждой строке.
    """
    norms = np.linalg.norm(embs, axis=1, keepdims=True)
    return embs / np.where(norms == 0, 1, norms)

In [10]:
def embed_queries(
    manager: ChromaDBManager,
    queries: List[str]
) -> np.ndarray:
    """
    Преобразует список текстовых запросов в нормализованные эмбеддинги.
    """
    embed_fn = manager.embedding_function
    q_embs = np.array(embed_fn(queries))  # shape (N_queries, dim)
    return normalize_embeddings(q_embs)

In [11]:
def compute_cosine_similarity(
    query_embs_norm: np.ndarray,
    col_embs_norm: np.ndarray
) -> np.ndarray:
    """
    Возвращает матрицу косинусного сходства (N_queries x N_docs).
    """
    return query_embs_norm.dot(col_embs_norm.T)


In [12]:
def get_topk_hits(
    sims: np.ndarray,
    docs: List[str],
    metas: List[dict],
    queries: List[str],
    top_k: int
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Для каждого запроса возвращает топ‑k документов с метаданными и схожестью.
    """
    results: Dict[str, List[Dict[str, Any]]] = {}
    for qi, query in enumerate(queries):
        sim_row = sims[qi]                       # вектор размера N_docs
        best_idxs = np.argsort(-sim_row)[:top_k]
        hits = []
        for idx in best_idxs:
            hits.append({
                'document':   docs[idx],
                'metadata':   metas[idx],
                'similarity': float(sim_row[idx])
            })
        results[query] = hits
    return results


In [13]:
def manual_search_by_embedding(
    collection_name: str,
    queries: List[str],
    top_k: int = 5
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Полный pipeline: 
     - загружает коллекцию, 
     - получает эмбеддинги запросов, 
     - считает косинусное сходство, 
     - возвращает топ‑k результатов для каждого запроса.
    """
    manager = ChromaDBManager()

    # 1) загрузка коллекции
    contents = load_collection_contents(manager, collection_name)
    col_embs_norm = normalize_embeddings(contents['embeddings'])

    # 2) эмбеддинги запросов
    q_embs_norm = embed_queries(manager, queries)

    # 3) матрица сходства
    sims = compute_cosine_similarity(q_embs_norm, col_embs_norm)

    # 4) сбор результатов
    return get_topk_hits(
        sims, 
        contents['documents'], 
        contents['metadatas'], 
        queries, 
        top_k
    )

In [14]:
queries = [
    "общая информация о конференции, правила оформления статей, требования к структуре и форматированию, цели"
]
output = manual_search_by_embedding("YSC", queries, top_k=3)

# Печать результатов
for q, hits in output.items():
    print(f"\n=== Запрос: {q} ===")
    for i, h in enumerate(hits, 1):
        print(f"{i}) sim={h['similarity']:.4f}, meta={h['metadata']}")
        print(h['document'])
        print("-" * 50)

2025-04-22 20:12:50.318 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-22 20:12:50.318 | INFO     | __main__:__init__:18 - Initializing ChromaDBManager with host: localhost, port: 8080
2025-04-22 20:12:50.562 | INFO     | __main__:__init__:23 - Successfully connected to ChromaDB
2025-04-22 20:12:50.562 | INFO     | __main__:__init__:37 - Created embedding function with model: BAAI/bge-m3
2025-04-22 20:12:50.565 | INFO     | __main__:get_collection:101 - Retrieved collection 'YSC'



=== Запрос: общая информация о конференции, правила оформления статей, требования к структуре и форматированию, цели ===
1) sim=0.5684, meta={'column_title': 'Aims and scope'}
Launched in 2010, Procedia Computer Science disseminates high quality conference proceedings across all topics of Computer Science research. Selected conference proceedings are published open access in a dedicated online Procedia volume on ScienceDirect.

Conference proceedings are selected for publication in Procedia Computer Science based on quality and are required to meet certain editorial criteria, including quality of the conference, relevance to an international audience, editorial standards, and relevance to current topics in Computer Science. Please see the instructions on how to prepare and submit a conference proceedings proposal at https://www.elsevier.com/physical-sciences-and-engineering/computer-science/journals/how-to-prepare-a-conference-proceeding-proposal-for-procedia

After a Conference Propo

### 2. Получение данных на основе текста статьи.

2.1 чанки (берём вариант 2.2)

In [15]:
def split_text_to_chunks(
    text: str,
    max_chunk_size: int = 500,
    overlap: int = 50
) -> List[str]:
    """
    Разбивает текст на перекрывающиеся чанки по словам.
    """
    words = text.split()
    chunks = []
    step = max_chunk_size - overlap
    for i in range(0, len(words), step):
        chunks.append(" ".join(words[i: i + max_chunk_size]))
    return chunks

2.2 Выделение темы, текста и литературы.

In [20]:
import re
from typing import List, Dict, Any
from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fortn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
def extract_title(text: str) -> str:
    """
    Извлекает название статьи как первую непустую строку.
    """
    for line in text.splitlines():
        stripped = line.strip()
        if stripped:
            return stripped
    return ""

def extract_literature_section(text: str) -> Tuple[str, str]:
    """
    Разделяет текст на основную часть и раздел 'Литература' (или 'References').
    Возвращает кортеж (main_text, literature_text).
    """
    pattern = re.compile(r'(^|\n)(литература|references)\s*[:\-]?\s*', re.IGNORECASE)
    match = pattern.search(text)
    if not match:
        return text.strip(), ""
    main = text[: match.start()].strip()
    literature = text[match.end():].strip()
    return main, literature

def preprocess_article(text: str) -> Dict[str, Any]:
    """
    Возвращает JSON-словарь со следующими ключами:
      - 'topic'     : полное название статьи (первая строка исходного текста)
      - 'text'      : основной текст без названия и раздела литературы
      - 'literature': текст раздела 'Литература', если он есть
    """
    # 1. Извлекаем название
    title = extract_title(text)

    # 2. Отделяем литературу от остального
    main_with_title, literature = extract_literature_section(text)

    # 3. Убираем название из начала основного текста, если оно там повторяется
    if main_with_title.startswith(title):
        main_text = main_with_title[len(title):].strip()
    else:
        main_text = main_with_title

    return np.array([title, main_text, literature])

def aggregate_hits_by_similarity(
    hits_by_chunk: Dict[str, List[Dict[str, Any]]],
    top_n: Optional[int] = None
) -> List[Dict[str, Any]]:
    """
    Группирует одинаковые документы из разных чанков, суммирует их similarity
    и возвращает список хитов, отсортированный по суммарной similarity.
    
    Args:
      hits_by_chunk: {
        chunk_text1: [ { 'document': str, 'metadata': dict, 'similarity': float }, ... ],
        chunk_text2: [ ... ],
        ...
      }
      top_n: если задано, возвращает только первые top_n элементов
    
    Возвращает:
      [
        {
          'document': str,
          'metadata': dict,
          'sum_similarity': float,  # сумма similarity из всех чанков
          'count': int,             # сколько раз встретился этот документ
          'chunks': List[str]       # список чанков, в которых найден
        },
        ...
      ]
    """
    agg: Dict[str, Dict[str, Any]] = {}

    for chunk, hits in hits_by_chunk.items():
        for hit in hits:
            doc = hit['document']
            sim = hit['similarity']
            md  = hit['metadata']

            if doc not in agg:
                agg[doc] = {
                    'document': doc,
                    'metadata': md,
                    'sum_similarity': sim,
                    'count': 1,
                    'chunks': [chunk]
                }
            else:
                agg_entry = agg[doc]
                agg_entry['sum_similarity'] += sim
                agg_entry['count'] += 1
                agg_entry['chunks'].append(chunk)

    # Преобразуем в список и сортируем по убыванию sum_similarity
    aggregated_list = sorted(
        agg.values(),
        key=lambda x: x['sum_similarity'],
        reverse=True
    )

    if top_n is not None:
        return aggregated_list[:top_n]
    return aggregated_list

Основаная часть

In [56]:
# Пример использования
article_path = "D:/GitHub/Edulytica/src/rag/data/article_2.txt"
with open(article_path, 'r', encoding='utf-8') as f:
    article_text = f.read()

In [ ]:
#chunks = split_text_to_chunks(article_text, max_chunk_size=500, overlap=50)

In [57]:
chunks = preprocess_article(article_text)

In [58]:
chunks

array(['РАЗРАБОТКА РЕЦЕПТУР БЕЗГЛЮТЕНОВЫХ МАКАРОННЫХ ИЗДЕЛИЙ',
       'Йимер Г.А.1 \nНаучный руководитель – к.т.н., доцент Баракова Н.В.1 \n1 – Университет ИТМО \nАннотация \nИзучены свойства макаронного теста, приготовленного из 40 % кукурузной муки, 15 % нутовой \nмуки и 15 % нативного и модифицированного кукурузного крахмала E1422 GranStarch MС. Было \nустановлено, что применение модифицированного крахмала в качестве структурообразователя \nдало положительные результаты - тесто приобрело эластичность, стало более растяжимым.  \nКлючевые слова \nБезглютеновые макаронные изделия, кукурузная мука, нутовая муки, модифицированный \nкукурузный крахмал \nВведение. Макаронные изделия относятся к продуктам массового потребления и \nпроизводятся из пшеницы твердых сортов. В состав пшеничной муки входит белок-глютен, \nкоторый и создает структуру теста. Мука, используемая для приготовления безглютеновых \nпродуктов, не содержит глютен. Обычно при разработке безглютеновых макаронных \nизделий и

In [53]:
# ищем по каждому чанку в коллекции "KMU"
results = manual_search_by_embedding("KMU", chunks, top_k=5)

# пример вывода
for chunk, hits in results.items():
    print(f"\n=== Chunk ===\n{chunk}\n")
    for i, h in enumerate(hits, 1):
        print(f"{i}) sim={h['similarity']:.4f}, meta={h['metadata']}")
        print(h['document'])
        print("-" * 40)

2025-04-22 22:58:51.389 | INFO     | rag.core.utils.config_loader:load_config:47 - Configuration loaded successfully: {'count_keywords': 10, 'host': 'localhost', 'port': 8080, 'collections_file': 'all_collections.json', 'embedding_model': 'BAAI/bge-m3'}
2025-04-22 22:58:51.390 | INFO     | __main__:__init__:18 - Initializing ChromaDBManager with host: localhost, port: 8080
2025-04-22 22:58:51.636 | INFO     | __main__:__init__:23 - Successfully connected to ChromaDB
2025-04-22 22:58:51.638 | INFO     | __main__:__init__:37 - Created embedding function with model: BAAI/bge-m3
2025-04-22 22:58:51.639 | INFO     | __main__:get_collection:101 - Retrieved collection 'KMU'



=== Chunk ===
РАЗРАБОТКА РЕЦЕПТУР БЕЗГЛЮТЕНОВЫХ МАКАРОННЫХ ИЗДЕЛИЙ

1) sim=0.4580, meta={'column_title': 'Секции__Науки о жизни__Infochemistry and Interdisciplinary research / Инфохимия и междисциплинарные исследования'}
К представлению принимаются междисциплинарные проекты, нацеленные на решение комплексных задач в области химии, материаловедения и смежных направлений.
Представляемые проекты могут быть ориентированы как на фундаментальные исследования в области разработки новых веществ, материалов и подходов, так и на оптимизацию существующих решений для автоматизации процессов.
Особое внимание уделяется проектам, в которых для обработки и систематизации данных используются информационные технологии и инструменты из области искусственного интеллекта.
К представлению также принимаются исследования, включающие:
моделирование веществ и материалов, их свойств и путей взаимодействия на различных уровнях;
разработку роботизированных и автоматизированных платформ для реализации химических п

In [59]:
# агрегируем и берём топ‑3
agg = aggregate_hits_by_similarity(results, top_n=1)

for i, item in enumerate(agg, 1):
    print(f"{i}) sum_sim={item['sum_similarity']:.4f}, count={item['count']}")
    print(" metadata:", item['metadata'])
    print(" document:", item['document'])
    print(" chunks:", item['chunks'])
    print("-"*50)

1) sum_sim=1.4534, count=3
 metadata: {'column_title': 'Секции__Науки о жизни__Infochemistry and Interdisciplinary research / Инфохимия и междисциплинарные исследования'}
 document: К представлению принимаются междисциплинарные проекты, нацеленные на решение комплексных задач в области химии, материаловедения и смежных направлений.
Представляемые проекты могут быть ориентированы как на фундаментальные исследования в области разработки новых веществ, материалов и подходов, так и на оптимизацию существующих решений для автоматизации процессов.
Особое внимание уделяется проектам, в которых для обработки и систематизации данных используются информационные технологии и инструменты из области искусственного интеллекта.
К представлению также принимаются исследования, включающие:
моделирование веществ и материалов, их свойств и путей взаимодействия на различных уровнях;
разработку роботизированных и автоматизированных платформ для реализации химических процессов.
К участию приглашаются аспиран

### 3. Получение данных на основе Промта.